# GitHub Connector

In [4]:
import re
import github
import getpass

## Preamble

## Connector

In [5]:


class GitHubConnector:
    def __init__(self, access_token):
        self.access_token = access_token
        self.github = github.Github(self.access_token)

    def _parse_repo_url(self, url):
        repo_pattern = r"github.com/([^/]+)/(.+)"
        match = re.match(repo_pattern, url)
        if match:
            return match.groups()
        else:
            return None

    def get_issues(self, repo_url, state="all"):
        repo_info = self._parse_repo_url(repo_url)
        if repo_info:
            owner, repo_name = repo_info
        else:
            print("Invalid repository URL")
            return

        # Get the repository object
        repo = self.github.get_repo(f"{owner}/{repo_name}")

        # Retrieve all issues
        issues = repo.get_issues(state=state)

        return list(issues)


In [6]:
# Example usage
access_token = getpass.getpass("Enter your GitHub access token: ")
github_connector = GitHubConnector(access_token)


Invalid repository URL


TypeError: 'NoneType' object is not iterable

In [ ]:
url = input("Enter the repository URL: ")
issues = github_connector.get_issues(url)
for issue in issues:
    print(issue.number, issue.title)